In [1]:
import os
os.environ["MKL_NUM_THREADS"] = '16'
os.environ["NUMEXPR_NUM_THREADS"] = '16'
os.environ["OMP_NUM_THREADS"] = '16'
print(os.environ.get('OMP_NUM_THREADS'))

16


In [2]:
import tensorflow as tf
import numpy as np
from PIL import Image
import rasterio
import imgaug as ia
from imgaug import augmenters as iaa
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

import imageio
import os

import time
import rasterio.warp             # Reproject raster samples
from functools import reduce
from tensorflow.keras.models import load_model

from core.UNet import UNet  #
from core.losses import tversky, focalTversky, bce_dice_loss, accuracy, dice_loss, IoU, recall, precision
from tensorflow.keras.losses import BinaryCrossentropy as bce
from core.optimizers import adaDelta, adagrad, adam, nadam
from core.frame_info import FrameInfo
from core.dataset_generator import DataGenerator
from core.visualize import display_images,plot
import json
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import shutil
import pickle
import random

%matplotlib inline
import matplotlib.pyplot as plt  # plotting tools
import matplotlib.patches as patches
from matplotlib.patches import Polygon
#matplotlib.use("Agg")

import warnings                  # ignore annoying warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Mixed precision is the use of both 16-bit and 32-bit floating-point types in a model during training to make it run faster and use less memory.
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
print(tf.__version__)

2.9.0


In [3]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto(
    #device_count={"CPU": 64},
    allow_soft_placement=True, 
    log_device_placement=False)
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [7]:
base_dir = r'D:\lakemapping\U_Net'
dataset_dir=r'D:\lakemapping\2_dataset'
image_type = '.png'       
NDWI_fn = 'ndwi'
red_fn = 'red'
blue_fn = 'blue'
green_fn = 'green'
swir_fn = 'swir'
annotation_fn = 'annotation'
type_num=6
patch_size = (512,512,6) 
patch_dir = os.path.join(base_dir,'patches{}'.format(patch_size[0])) 
# The training areas are divided into training, validation and testing set. Note that training area can have different sizes, so it doesn't guarantee that the final generated patches (when using sequential stratergy) will be in the same ratio.
test_ratio = 0.2
val_ratio = 0.25

## 划分数据集
从output文件夹存储到patchReshape文件夹，train和test按类别type分别存储

In [8]:
#hbh: check whether the output dir(must be present) of each type is empty
for i in range(0,type_num):
    path_to_write=os.path.join(dataset_dir,'output550\output{}'.format(i))
    assert os.path.exists(path_to_write)
    if len(os.listdir(path_to_write))==0:
         print('Warning: path_to_write{} is empty!'.format(i))

In [9]:
def removeFileinType(lakeType,patchType,fn):
    output_dir=os.path.join(dataset_dir,'patchesReshape550\\{}\\type{}'.format(patchType,lakeType))
    remove(output_dir,fn)
    remove(output_dir,fn.replace(NDWI_fn,annotation_fn))
    remove(output_dir,fn.replace(NDWI_fn,red_fn))
    remove(output_dir,fn.replace(NDWI_fn,green_fn))
    remove(output_dir,fn.replace(NDWI_fn,blue_fn))
    remove(output_dir,fn.replace(NDWI_fn,swir_fn))
    
def remove(output_dir,fn):
    new=os.path.join(output_dir,fn)
    old=os.path.join(patch_output,fn)
    shutil.copy(old, new)

In [10]:
# split dataset and remove images & annotations to new directory
for i in range(0,type_num):
    frames = []
    frames_json = os.path.join(patch_dir,'frames_list{}.json'.format(i))
    patch_output = os.path.join(dataset_dir,'output550\output{}'.format(i))
    all_files = os.listdir(patch_output)
    all_files_NDWI = [fn for fn in all_files if fn.startswith(NDWI_fn) and fn.endswith(image_type)]#ndwi.png
    len(all_files_NDWI)
    print(all_files_NDWI)

    if os.path.isfile(frames_json):
            print("images had been removed")
    else:
            print("Creating and writing train-test split from file")
            frames_list = list(range(len(all_files_NDWI)))
            
            # Divide into training and test set       
            training_frames, testing_frames = train_test_split(frames_list, test_size=test_ratio)
            # Further divide into training set into training and validataion set              
            training_frames, validation_frames = train_test_split(training_frames, test_size=val_ratio)
            
            training_frames_name=[all_files_NDWI[id] for id in training_frames]
            testing_frames_name=[all_files_NDWI[id] for id in testing_frames]
            validation_frames_name=[all_files_NDWI[id] for id in validation_frames]
            
            #train data are classified into different dirs for following process.
            for train_fn in training_frames_name:
                removeFileinType(i,'train',train_fn)
            for testing_fn in testing_frames_name:
                removeFileinType(i,'test',testing_fn) 
            for validation_fn in validation_frames_name:
                removeFileinType(i,'val',validation_fn)
                
            frame_split = {
                'training_frames': training_frames_name,
                'testing_frames': testing_frames_name,
                'validation_frames': validation_frames_name
            }
            
            if not os.path.exists(patch_dir):
                os.makedirs(patch_dir)
            with open(frames_json, 'w') as f:
                json.dump(frame_split, f)
                
            print('training_frames', training_frames_name)
            print('validation_frames',validation_frames_name )
            print('testing_frames', testing_frames_name)

195

['ndwi_100.png', 'ndwi_101.png', 'ndwi_106.png', 'ndwi_107.png', 'ndwi_108.png', 'ndwi_113.png', 'ndwi_120.png', 'ndwi_123.png', 'ndwi_126.png', 'ndwi_132.png', 'ndwi_136.png', 'ndwi_140.png', 'ndwi_142.png', 'ndwi_144.png', 'ndwi_15.png', 'ndwi_151.png', 'ndwi_152.png', 'ndwi_153.png', 'ndwi_154.png', 'ndwi_158.png', 'ndwi_164.png', 'ndwi_165.png', 'ndwi_166.png', 'ndwi_168.png', 'ndwi_169.png', 'ndwi_17.png', 'ndwi_170.png', 'ndwi_171.png', 'ndwi_177.png', 'ndwi_178.png', 'ndwi_181.png', 'ndwi_19.png', 'ndwi_229.png', 'ndwi_230.png', 'ndwi_231.png', 'ndwi_232.png', 'ndwi_235.png', 'ndwi_236.png', 'ndwi_237.png', 'ndwi_238.png', 'ndwi_239.png', 'ndwi_241.png', 'ndwi_247.png', 'ndwi_251.png', 'ndwi_253.png', 'ndwi_255.png', 'ndwi_259.png', 'ndwi_26.png', 'ndwi_265.png', 'ndwi_268.png', 'ndwi_269.png', 'ndwi_27.png', 'ndwi_270.png', 'ndwi_271.png', 'ndwi_272.png', 'ndwi_276.png', 'ndwi_277.png', 'ndwi_278.png', 'ndwi_279.png', 'ndwi_28.png', 'ndwi_280.png', 'ndwi_282.png', 'ndwi_286.png

163

['ndwi_0.png', 'ndwi_10.png', 'ndwi_102.png', 'ndwi_103.png', 'ndwi_104.png', 'ndwi_105.png', 'ndwi_11.png', 'ndwi_111.png', 'ndwi_112.png', 'ndwi_114.png', 'ndwi_115.png', 'ndwi_117.png', 'ndwi_12.png', 'ndwi_121.png', 'ndwi_128.png', 'ndwi_129.png', 'ndwi_130.png', 'ndwi_133.png', 'ndwi_134.png', 'ndwi_135.png', 'ndwi_145.png', 'ndwi_156.png', 'ndwi_16.png', 'ndwi_160.png', 'ndwi_172.png', 'ndwi_173.png', 'ndwi_18.png', 'ndwi_185.png', 'ndwi_2.png', 'ndwi_21.png', 'ndwi_22.png', 'ndwi_228.png', 'ndwi_233.png', 'ndwi_234.png', 'ndwi_24.png', 'ndwi_242.png', 'ndwi_243.png', 'ndwi_248.png', 'ndwi_254.png', 'ndwi_256.png', 'ndwi_258.png', 'ndwi_261.png', 'ndwi_263.png', 'ndwi_264.png', 'ndwi_266.png', 'ndwi_267.png', 'ndwi_273.png', 'ndwi_274.png', 'ndwi_275.png', 'ndwi_281.png', 'ndwi_283.png', 'ndwi_285.png', 'ndwi_3.png', 'ndwi_300.png', 'ndwi_301.png', 'ndwi_304.png', 'ndwi_308.png', 'ndwi_311.png', 'ndwi_317.png', 'ndwi_318.png', 'ndwi_32.png', 'ndwi_321.png', 'ndwi_331.png', 'ndwi_

88

...

['ndwi_1.png', 'ndwi_110.png', 'ndwi_13.png', 'ndwi_14.png', 'ndwi_141.png', 'ndwi_187.png', 'ndwi_188.png', 'ndwi_189.png', 'ndwi_190.png', 'ndwi_191.png', 'ndwi_192.png', 'ndwi_193.png', 'ndwi_194.png', 'ndwi_195.png', 'ndwi_196.png', 'ndwi_197.png', 'ndwi_198.png', 'ndwi_199.png', 'ndwi_200.png', 'ndwi_201.png', 'ndwi_202.png', 'ndwi_203.png', 'ndwi_204.png', 'ndwi_205.png', 'ndwi_206.png', 'ndwi_207.png', 'ndwi_208.png', 'ndwi_209.png', 'ndwi_210.png', 'ndwi_211.png', 'ndwi_212.png', 'ndwi_213.png', 'ndwi_214.png', 'ndwi_215.png', 'ndwi_216.png', 'ndwi_217.png', 'ndwi_218.png', 'ndwi_219.png', 'ndwi_220.png', 'ndwi_221.png', 'ndwi_222.png', 'ndwi_223.png', 'ndwi_224.png', 'ndwi_225.png', 'ndwi_226.png', 'ndwi_227.png', 'ndwi_23.png', 'ndwi_240.png', 'ndwi_252.png', 'ndwi_284.png', 'ndwi_319.png', 'ndwi_322.png', 'ndwi_350.png', 'ndwi_4.png', 'ndwi_426.png', 'ndwi_435.png', 'ndwi_476.png', 'ndwi_498.png', 'ndwi_5.png', 'ndwi_501.png', 'ndwi_502.png', 'ndwi_503.png', 'ndwi_504.png', 

16

['ndwi_109.png', 'ndwi_118.png', 'ndwi_119.png', 'ndwi_124.png', 'ndwi_125.png', 'ndwi_162.png', 'ndwi_246.png', 'ndwi_249.png', 'ndwi_25.png', 'ndwi_366.png', 'ndwi_376.png', 'ndwi_378.png', 'ndwi_392.png', 'ndwi_407.png', 'ndwi_430.png', 'ndwi_53.png']
Creating and writing train-test split from file
training_frames ['ndwi_430.png', 'ndwi_378.png', 'ndwi_124.png', 'ndwi_249.png', 'ndwi_162.png', 'ndwi_53.png', 'ndwi_407.png', 'ndwi_109.png', 'ndwi_125.png']
validation_frames ['ndwi_366.png', 'ndwi_25.png', 'ndwi_246.png']
testing_frames ['ndwi_118.png', 'ndwi_392.png', 'ndwi_119.png', 'ndwi_376.png']


21

['ndwi_131.png', 'ndwi_138.png', 'ndwi_139.png', 'ndwi_146.png', 'ndwi_147.png', 'ndwi_148.png', 'ndwi_149.png', 'ndwi_183.png', 'ndwi_184.png', 'ndwi_244.png', 'ndwi_245.png', 'ndwi_250.png', 'ndwi_257.png', 'ndwi_310.png', 'ndwi_315.png', 'ndwi_390.png', 'ndwi_391.png', 'ndwi_408.png', 'ndwi_431.png', 'ndwi_432.png', 'ndwi_433.png']
Creating and writing train-test split from file
training_frames ['ndwi_184.png', 'ndwi_183.png', 'ndwi_149.png', 'ndwi_310.png', 'ndwi_147.png', 'ndwi_131.png', 'ndwi_432.png', 'ndwi_315.png', 'ndwi_250.png', 'ndwi_408.png', 'ndwi_431.png', 'ndwi_433.png']
validation_frames ['ndwi_146.png', 'ndwi_391.png', 'ndwi_257.png', 'ndwi_139.png']
testing_frames ['ndwi_390.png', 'ndwi_138.png', 'ndwi_244.png', 'ndwi_245.png', 'ndwi_148.png']
